# Assignment 03 Part A - FastAI

## Assignment Description

Perform the following tasks:
Use FastAI to write a detailed Colab notebook that shows training and inference for the following tasks using FastAI. Showcase results and metrics:

- Tabular task


### Setup

In [1]:
!pip install fastai
!pip install ucimlrepo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00


In [2]:
from fastai.vision.all import *
from fastai.tabular.all import *
from fastai.text.all import *
from fastai.vision import *
from fastai.collab import *
from io import BytesIO
import pandas as pd
import requests
import os

In [3]:
pip install --upgrade git+https://github.com/rbrtwlz/fastai_object_detection

  Cloning https://github.com/rbrtwlz/fastai_object_detection to /tmp/pip-req-build-ws90mbw6
  Running command git clone --filter=blob:none --quiet https://github.com/rbrtwlz/fastai_object_detection /tmp/pip-req-build-ws90mbw6
  Resolved https://github.com/rbrtwlz/fastai_object_detection to commit 8935053a856e1bbaa793f9b9abb3922365ef3c8d
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/bes-dev/mean_average_precision to /tmp/pip-install-5l_kvozb/mean-average-precision_fd5906ac7d954f0e98cb72528889f5da
  Running command git clone --filter=blob:none --quiet https://github.com/bes-dev/mean_average_precision /tmp/pip-install-5l_kvozb/mean-average-precision_fd5906ac7d954f0e98cb72528889f5da
  Resolved https://github.com/bes-dev/mean_average_precision to commit 477749cf84cb7136f38974c454ee7206df447dc5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21

# **Tabular Task**

In [2]:
from fastai.tabular.all import *

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/adult.csv')]

In [4]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [5]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [6]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [7]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [8]:
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
2653,5,13,3,5,1,5,1,-0.117181,0.269542,1.539424
25566,5,16,5,13,2,2,1,-1.068945,-1.101849,-0.028976


In [9]:
dls = to.dataloaders(bs=64)

In [20]:
learn = tabular_learner(dls, metrics=accuracy)

In [22]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.362097,0.375975,0.827396,00:05


In [23]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,6.0,13.0,3.0,11.0,6.0,5.0,1.0,0.029244,0.122420,1.539424,1.0,1.0
1,7.0,16.0,3.0,5.0,1.0,5.0,1.0,1.713135,-0.632131,-0.028976,0.0,1.0
2,5.0,9.0,3.0,4.0,1.0,5.0,1.0,-1.142158,-1.402593,0.363124,1.0,0.0
3,5.0,12.0,6.0,2.0,4.0,5.0,1.0,-0.702882,-0.278897,-0.421076,0.0,0.0
4,5.0,10.0,3.0,5.0,1.0,2.0,1.0,-0.483244,1.338922,1.147324,1.0,1.0
5,7.0,10.0,5.0,11.0,2.0,5.0,1.0,-0.776095,0.879506,1.147324,0.0,0.0
6,5.0,9.0,5.0,10.0,2.0,5.0,1.0,0.395307,-1.162853,0.363124,0.0,0.0
7,5.0,12.0,3.0,8.0,1.0,5.0,1.0,-0.922520,-0.714305,-0.421076,0.0,0.0
8,5.0,10.0,3.0,5.0,1.0,3.0,1.0,0.322095,2.019321,1.147324,1.0,1.0


In [24]:
row, clas, probs = learn.predict(df.iloc[0])

In [25]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.003035,12.0,>=50k


In [26]:
clas, probs

(tensor(1), tensor([0.4488, 0.5512]))

In [27]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [28]:
learn.get_preds(dl=dl)

(tensor([[0.4488, 0.5512],
         [0.5110, 0.4890],
         [0.9272, 0.0728],
         ...,
         [0.5977, 0.4023],
         [0.7220, 0.2780],
         [0.6361, 0.3639]]),
 None)